In [1]:
from transformers import AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import pandas as pd
import pandas as pd
from datasets import Dataset, load_dataset, load_from_disk
import optuna
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
import torch
from transformers import AutoTokenizer
import pandas as pd
from datasets import Dataset, load_dataset, load_from_disk
from transformers import AutoTokenizer

[2025-06-04 18:31:00,476] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


W0604 18:31:02.292000 27400 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", 'en')
dataset = dataset['train'].train_test_split(test_size=0.1)
df = pd.concat([dataset['train'].to_pandas(), dataset['test'].to_pandas()], ignore_index=True)
df = df.drop_duplicates(subset=['Question'])
print(df.head())
print(len(df))


                                            Question  \
0  What is the mechanism of analgesia in terms of...   
1  Considering a 5-year-old boy who presents with...   
2  A 70-year-old male presents with excessive fat...   
3  A 3-year-old boy presents with intermittent ab...   
4  A 57-year-old man presents with a painful, swo...   

                                         Complex_CoT  \
0  Okay, so let’s think about how pain relief act...   
1  Okay, so we have a little boy, just five years...   
2  Okay, let's see what we've got here. There's a...   
3  I'm looking at a 3-year-old boy who has abdomi...   
4  So, a 57-year-old guy comes in with his big to...   

                                            Response  
0  Pain relief, or analgesia, involves various re...  
1  Given the symptoms and the normal ultrasound r...  
2  Based on the provided information, the agent c...  
3  The symptoms and findings you describe in the ...  
4  Based on the symptoms described—sudden onset o..

## Creating clusters to obtain significant subset of dataset

In [3]:
# Prepare the text data
texts = df["Question"].astype(str).tolist()

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1024)
X = vectorizer.fit_transform(texts)

# KMeans clustering
n_clusters = 1968
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(X)

# For each cluster, find the question closest to the centroid
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
centroid_questions = [texts[i] for i in closest]

# Remove duplicates to ensure no repeated questions
unique_centroid_questions = list(dict.fromkeys(centroid_questions))

# Create a DataFrame with the unique centroid questions
centroids_df = pd.DataFrame({'centroid_question': unique_centroid_questions})
print(centroids_df.head())
print(f"Number of unique centroid questions: {len(centroids_df)}")

                                   centroid_question
0  A 50-year-old chronic smoker presents with his...
1  67岁，男，呼吸急促3个月，请结合胸片选出最可能的诊断  (    )<br />[img=...
2  A 42-year-old woman comes to the physician bec...
3  A 24- year old man falls on the ground when he...
4  A 55-year-old man presents with well-being sen...
Number of unique centroid questions: 1250


In [4]:
# Merge centroid questions with original df to get their corresponding responses
centroids_with_responses = pd.merge(
    centroids_df,
    df[['Question', 'Response']],
    left_on='centroid_question',
    right_on='Question',
    how='left'
)[['Question', 'Response']]

centroids_with_responses = centroids_with_responses.drop_duplicates(subset=['Question'])

print(centroids_with_responses.head())
print(centroids_with_responses.shape)

                                            Question  \
0  A 50-year-old chronic smoker presents with his...   
1  67岁，男，呼吸急促3个月，请结合胸片选出最可能的诊断  (    )<br />[img=...   
2  A 42-year-old woman comes to the physician bec...   
3  A 24- year old man falls on the ground when he...   
4  A 55-year-old man presents with well-being sen...   

                                            Response  
0  Based on the scenario provided, the best optio...  
1  Based on the information provided, the most li...  
2  The clinical picture points toward Primary Bil...  
3  In this scenario, the young man is exhibiting ...  
4  Given the symptoms described, such as hyperten...  
(1250, 2)


In [5]:
# Convert DataFrame to Hugging Face Dataset
centroids_dataset = Dataset.from_pandas(centroids_with_responses, preserve_index=False)

# Save to disk (as Arrow format)
centroids_dataset.save_to_disk("centroids_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1250 [00:00<?, ? examples/s]

In [6]:
# To load it later:

loaded_dataset = load_from_disk("centroids_dataset")
print(loaded_dataset.to_pandas().head())
print(loaded_dataset.to_pandas().shape)
print(loaded_dataset)

                                            Question  \
0  A 50-year-old chronic smoker presents with his...   
1  67岁，男，呼吸急促3个月，请结合胸片选出最可能的诊断  (    )<br />[img=...   
2  A 42-year-old woman comes to the physician bec...   
3  A 24- year old man falls on the ground when he...   
4  A 55-year-old man presents with well-being sen...   

                                            Response  
0  Based on the scenario provided, the best optio...  
1  Based on the information provided, the most li...  
2  The clinical picture points toward Primary Bil...  
3  In this scenario, the young man is exhibiting ...  
4  Given the symptoms described, such as hyperten...  
(1250, 2)
Dataset({
    features: ['Question', 'Response'],
    num_rows: 1250
})


In [7]:
loaded_dataset = loaded_dataset.train_test_split(test_size=0.1)
print(len(loaded_dataset["train"]))  # Number of rows in the train split
print(len(loaded_dataset["test"]))   # Number of rows in the test split (if exists)
print(loaded_dataset["test"].to_pandas().head())

1125
125
                                            Question  \
0  An 8-month old child presented with itchy, exu...   
1  What is the best treatment approach for a 12-y...   
2  In complete 3\nrd \nNerve palsy-a) Eye Deviate...   
3  A 34-year-old woman presents with decreased ap...   
4  A patient presents with malignant hypehermia a...   

                                            Response  
0  The best treatment choice for an 8-month-old c...  
1  The best treatment approach for a 12-year-old ...  
2  In the case of complete 3rd nerve palsy, the i...  
3  Based on the symptoms you've described—decreas...  
4  The immediate treatment for malignant hyperthe...  


In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"  # Base model, replace with your preferred Mistral version
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

In [9]:
# Function to tokenize and format the data
def preprocess_function(examples):
    # Format: <s>[INST] {instruction} [/INST] {response}</s>
    prompts = [f"[INST] {question} [/INST]" for question in examples['Question']]
    responses = examples['Response']
    
    # Combine prompts and responses
    texts = []
    for prompt, response in zip(prompts, responses):
        texts.append(f"{prompt} {response}")
    
    print(texts[:1])  # Print first 5 formatted texts for debugging
    # Tokenize
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,  # Adjust based on your needs
        padding="max_length",
        return_tensors="pt"
    )
    
    # Prepare labels for the decoder (same as input_ids for causal language modeling)
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

In [10]:
tokenized_dataset = loaded_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=loaded_dataset["train"].column_names,
)

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

['[INST] What are the characteristics and associations of membranous glomerulonephritis? [/INST] Membranous glomerulonephritis, also known as membranous nephropathy, is a kidney disorder characterized by the thickening of the walls of the glomerular capillaries due to the deposition of immune complexes. It can be classified into primary and secondary types.\n\n**Primary Membranous Nephropathy:**\n- The primary form is often idiopathic but has been linked to the presence of antibodies against the phospholipase A2 receptor (PLA2R) on podocytes. This immune response plays a pivotal role in disease pathogenesis. \n\n**Secondary Membranous Nephropathy:**\n- Secondary membranous nephropathy is associated with other conditions, including:\n  - Infections like hepatitis B and C.\n  - Medications such as NSAIDs and penicillamine.\n  - Certain cancers.\n  - Autoimmune diseases, notably systemic lupus erythematosus.\n\n**Clinical Presentation:**\n- The hallmark clinical presentation is nephrotic 

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

['[INST] An 8-month old child presented with itchy, exudative lesions on the face, palms and soles. The siblings also have similar complaints. The treatment of choice is such a patient is:\nA. Systemic ampicillin\nB. Topical betamethasone\nC. Systemic prednisolone\nD. Topical permethrin [/INST] The best treatment choice for an 8-month-old child with itchy, exudative lesions on the face, palms, and soles, particularly when siblings are also affected, is D. Topical permethrin. This option targets scabies, a highly contagious skin condition caused by mites, which aligns with the symptoms and spread among family members.']


In [11]:
print(tokenized_dataset["train"].to_pandas().head())

                                           input_ids  \
0  [1, 733, 16289, 28793, 1824, 460, 272, 15559, ...   
1  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   
2  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   
3  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   
4  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...   

                                      attention_mask  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                              labels  
0  [1, 733, 16289, 28793, 1824, 460, 272, 15559, ...  
1  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...  
2  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...  
3  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...  
4  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",  # Automatically distribute across available GPUs
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
def try_import_wandb():
    try:
        import wandb
        return True
    except ImportError:
        return False

In [14]:
def optuna_objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8])
    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 4)
    gradient_accumulation_steps = trial.suggest_categorical("gradient_accumulation_steps", [4, 8, 16])

    training_args = TrainingArguments(
        output_dir="./mistral-finetuned",
        run_name=f"trial_{trial.number}",
        overwrite_output_dir=True,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        save_steps=1000,
        save_total_limit=1,
        eval_steps=500,
        logging_dir="./logs",
        logging_steps=50,
        learning_rate=learning_rate,
        weight_decay=0.01,
        warmup_steps=200,
        lr_scheduler_type="cosine",
        report_to="wandb" if try_import_wandb() else "none",
        fp16=False,
        bf16=True,
        gradient_checkpointing=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
    )

    trainer.train()
    eval_metrics = trainer.evaluate()
    return eval_metrics["eval_loss"]  # or use another metric, e.g., "eval_accuracy"


In [15]:
# Run Optuna study
study = optuna.create_study(direction="minimize")  # minimize eval_loss
study.optimize(optuna_objective, n_trials=10)

print("Best trial:")
print(study.best_trial)
print("Best params:")
print(study.best_params)

[I 2025-06-04 18:34:15,030] A new study created in memory with name: no-name-78836935-4480-415f-9f09-1a2c000d57ce
wandb: Currently logged in as: luis-orellana777 (luis-orellana777-sngular) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss


[I 2025-06-04 20:22:31,247] Trial 0 finished with value: 0.4569173753261566 and parameters: {'learning_rate': 1.3566154061739588e-05, 'per_device_train_batch_size': 4, 'num_train_epochs': 1, 'gradient_accumulation_steps': 8}. Best is trial 0 with value: 0.4569173753261566.


Step,Training Loss
50,0.448300
100,0.445400
150,1.114500
200,0.417300
250,0.329400


...

Step,Training Loss
50,0.017000


[I 2025-06-06 04:04:10,049] Trial 8 finished with value: 1.19642972946167 and parameters: {'learning_rate': 1.0787161757541825e-05, 'per_device_train_batch_size': 4, 'num_train_epochs': 2, 'gradient_accumulation_steps': 8}. Best is trial 0 with value: 0.4569173753261566.


Step,Training Loss


[I 2025-06-06 07:26:49,336] Trial 9 finished with value: 1.2786601781845093 and parameters: {'learning_rate': 2.7181544659314106e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 2, 'gradient_accumulation_steps': 8}. Best is trial 0 with value: 0.4569173753261566.


Best trial:
FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.4569173753261566], datetime_start=datetime.datetime(2025, 6, 4, 18, 34, 15, 30780), datetime_complete=datetime.datetime(2025, 6, 4, 20, 22, 31, 246917), params={'learning_rate': 1.3566154061739588e-05, 'per_device_train_batch_size': 4, 'num_train_epochs': 1, 'gradient_accumulation_steps': 8}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-05, step=None), 'per_device_train_batch_size': CategoricalDistribution(choices=(2, 4, 8)), 'num_train_epochs': IntDistribution(high=4, log=False, low=1, step=1), 'gradient_accumulation_steps': CategoricalDistribution(choices=(4, 8, 16))}, trial_id=0, value=None)
Best params:
{'learning_rate': 1.3566154061739588e-05, 'per_device_train_batch_size': 4, 'num_train_epochs': 1, 'gradient_accumulation_steps': 8}
